## Gun violence in America—looking at data
A peek at some frequency distributions of variables in some interesting data sets.

In [1]:
# Imports
import pandas as pd
import numpy as np
import os # for filepath

#### US Mass Shootings, 1982-2018: Data From Mother Jones’ Investigation
https://www.motherjones.com/politics/2012/12/mass-shootings-mother-jones-full-data/f

In [2]:
# Loading the data
datapath = os.path.join('datasets', "")
mojo_df = pd.read_csv(datapath + 'mojo_us_mass_shoot_82_18.csv')

In [3]:
# A look at the columns and their data types
mojo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 22 columns):
Case                                   97 non-null object
Location                               97 non-null object
Date                                   97 non-null object
Year                                   97 non-null int64
Summary                                97 non-null object
Fatalities                             97 non-null int64
Injured                                97 non-null int64
Total victims                          97 non-null object
Venue                                  97 non-null object
Prior signs of mental health issues    97 non-null object
Mental health - details                86 non-null object
Weapons obtained legally               97 non-null object
Where obtained                         85 non-null object
Type of weapons                        97 non-null object
Weapon details                         91 non-null object
Race                          

In [4]:
# A look at the first few rows of data
mojo_df.head()

,Case,Location,Date,Year,Summary,Fatalities,Injured,Total victims,Venue,Prior signs of mental health issues,...,Where obtained,Type of weapons,Weapon details,Race,Gender,Sources,Mental Health Sources,latitude,longitude,Type
0,Stoneman Douglas High School shooting,"Parkland, Florida",2/14/18,2018,"Nikolas J. Cruz, 19, heavily armed with an AR-...",17,14,31,School,Yes,...,A Florida pawn shop,semiautomatic rifle,AR-15,White,M,https://www.nytimes.com/2018/02/14/us/parkland...,https://www.nytimes.com/2018/02/15/us/nikolas-...,NaN,NaN,Mass
1,Pennsylvania carwash shooting,"Melcroft, PA",1/28/18,2018,"Timothy O'Brien Smith, 28, wearing body armor ...",4,1,5,Other,TBD,...,TBD,semiautomatic rifle and semiautomatic handgun,NaN,White,M,http://www.wpxi.com/news/top-stories/family-me...,NaN,NaN,NaN,Mass
2,Rancho Tehama shooting spree,"Rancho Tehama, CA",11/14/17,2017,"Kevin Janson Neal, 44, went on an approximatel...",5,10,15,Other,TBD,...,TBD,semiautomatic rifles,Two illegally modified rifles,White,M,https://www.nbcnews.com/news/us-news/californi...,NaN,NaN,NaN,Spree
3,Texas First Baptist Church massacre,"Sutherland Springs, TX",11/5/17,2017,"Devin Patrick Kelley, a 26-year-old ex-US Air ...",26,20,46+,Religious,Yes,...,Purchased in April 2016 from an Academy Sports...,semiautomatic rifle,Ruger AR-556; Kelley also possessed semiautoma...,White,M,https://www.washingtonpost.com/news/morning-mi...,http://www.expressnews.com/news/local/article/...,32.780105,-96.800008,Mass
4,Walmart shooting in suburban Denver,"Thornton, CO",11/1/17,2017,"Scott Allen Ostrem, 47, walked into a Walmart ...",3,0,3,Other,Unclear,...,NaN,semiautomatic handgun,NaN,White,M,https://www.nytimes.com/2017/11/01/us/thornton...,NaN,43.060567,-88.106479,Mass
